In [3]:
import seaborn as sns

sns.set(rc={'axes.facecolor': 'dimgrey', 'grid.color': 'lightgrey'})
import torch
# Now train the DT classifier
# Load libraries
import pandas as pd
from sklearn.metrics import classification_report

In [4]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
%run utils.ipynb
%run gcn_model.ipynb
%run GAT_model.ipynb

In [6]:
# class 1 is illicit, 2 is licit, unknown is unknown
labels = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edges = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
nodes = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)
print('done')

done


In [7]:
indexes = labels.index[labels["class"] != "unknown"].tolist()
new_labels = labels.replace("unknown", "3")

In [8]:
import numpy as np


@torch.no_grad()
def test(model, data, loss_fn):
    model.eval()
    res = {}
    losses = []
    for i, batch in enumerate(data):
        out = model.forward(batch)
        y_true = batch.y
        indexes = y_true != 2

        y_pred = out.argmax(dim=-1, keepdim=True)[indexes].view(-1, 1).cpu().detach().numpy()
        y_true = y_true[indexes].view(-1, 1).cpu().detach().numpy()

        res = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    return res

In [9]:
def trainGCN(data):
    GCN = GNN_Example(node_dim=data['train'][0].x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3,
                      K=2, dropout_rate=0.1).to(device)

    lr = 1e-3
    epochs = 50
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GCN.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_test_loss = 10e9
    best_result = None
    train_losses = []
    test_losses = []
    result = None
    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GCN, data['train'], optimizer, loss_fn, device)
        train_losses.append(loss)
        result = test(GCN, data["test"],loss_fn)

        # if epoch % 10 == 0:
        #     print("Epoch: ", epoch)
        #     for item, amount in result.items():
        #         print("{} ({})".format(item, amount))
        #     print("############################")

    return result


In [10]:
# cross validation
results = []

for idx in range(49):
    print("Starting Index: ", idx)
    data, graph_info = time_step_split_cross_validation(nodes, edges, new_labels, device, test_index=idx)

    result = trainGCN(data)
    results.append((idx, result))
    print("Done")

Starting Index:  0
Done
Starting Index:  1
Done
Starting Index:  2
Done
Starting Index:  3
Done
Starting Index:  4
Done
Starting Index:  5
Done
Starting Index:  6
Done
Starting Index:  7
Done
Starting Index:  8
Done
Starting Index:  9
Done
Starting Index:  10
Done
Starting Index:  11
Done
Starting Index:  12
Done
Starting Index:  13
Done
Starting Index:  14
Done
Starting Index:  15
Done
Starting Index:  16
Done
Starting Index:  17
Done
Starting Index:  18
Done
Starting Index:  19
Done
Starting Index:  20
Done
Starting Index:  21
Done
Starting Index:  22
Done
Starting Index:  23
Done
Starting Index:  24
Done
Starting Index:  25
Done
Starting Index:  26
Done
Starting Index:  27
Done
Starting Index:  28
Done
Starting Index:  29
Done
Starting Index:  30
Done
Starting Index:  31
Done
Starting Index:  32
Done
Starting Index:  33
Done
Starting Index:  34
Done
Starting Index:  35
Done
Starting Index:  36
Done
Starting Index:  37
Done
Starting Index:  38
Done
Starting Index:  39
Done
Starting I

In [1]:
import pickle
print(results)
with open("results_cross_validation", "wb") as fp:   #Pickling
    pickle.dump(results, fp)

NameError: name 'results' is not defined

In [3]:
with open("results_cross_validation", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print(b)

[(0, {'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 17}, '1': {'precision': 0.9920560747663552, 'recall': 0.9967136150234742, 'f1-score': 0.9943793911007026, 'support': 2130}, 'accuracy': 0.9888216115510013, 'macro avg': {'precision': 0.4960280373831776, 'recall': 0.4983568075117371, 'f1-score': 0.4971896955503513, 'support': 2147}, 'weighted avg': {'precision': 0.9842009498147819, 'recall': 0.9888216115510013, 'f1-score': 0.9865058700719593, 'support': 2147}}), (1, {'0': {'precision': 0.16666666666666666, 'recall': 0.05555555555555555, 'f1-score': 0.08333333333333333, 'support': 18}, '1': {'precision': 0.9846984698469847, 'recall': 0.9954504094631483, 'f1-score': 0.9900452488687784, 'support': 1099}, 'accuracy': 0.9803043867502238, 'macro avg': {'precision': 0.5756825682568257, 'recall': 0.5255029825093519, 'f1-score': 0.5366892911010559, 'support': 1117}, 'weighted avg': {'precision': 0.971516220556702, 'recall': 0.9803043867502238, 'f1-score': 0.9754339556909467